In [13]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [14]:
class DogsVsCats():
      IMG_SIZE=50
      CATS="C:/Users/DHRUVRAJ/AppData/Local/Programs/Python/Python310/Scripts/PyTorch/.ipynb_checkpoints/kagglecatsanddogs_5340/PetImages/Cat"
      DOGS="C:/Users/DHRUVRAJ/AppData/Local/Programs/Python/Python310/Scripts/PyTorch/.ipynb_checkpoints/kagglecatsanddogs_5340/PetImages/Dog"
      LABELS={CATS:0,DOGS:1}
      training_data=[]
      catcount=0
      dogcount=0
      def make_training_data(self):
            for label in self.LABELS: #CATS AND DOGS
                 print(label) #CATS THEN DOGS
                 for f in tqdm(os.listdir(label)): #IMAGES IN CATS AND DOGS
                  try:
                      path=os.path.join(label,f) #JOINING IMG IN CATS AND DOGS
                      img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)#CONVERTING IMAGES TO GRAY SCALE
                      img=cv2.resize(img,(self.IMG_SIZE,self.IMG_SIZE))
                      self.training_data.append([np.array(img),np.eye(2)[self.LABELS[label]]])  
                      if label==self.CATS:
                            self.catcount+=1
                      if label==self.DOGS:
                            self.dogcount+=1
                  except Exception as e:
                       pass
            np.random.shuffle(self.training_data)
            print("Cats: "+str(self.catcount))
            print("Dogs: "+str(self.dogcount))
            np.save("training_data.npy",self.training_data,allow_pickle=True,fix_imports=True)
                   
dogsvscats=DogsVsCats()
dogsvscats.make_training_data()

C:/Users/DHRUVRAJ/AppData/Local/Programs/Python/Python310/Scripts/PyTorch/.ipynb_checkpoints/kagglecatsanddogs_5340/PetImages/Cat


  0%|          | 0/12501 [00:00<?, ?it/s]

100%|██████████| 12501/12501 [00:32<00:00, 385.77it/s]


C:/Users/DHRUVRAJ/AppData/Local/Programs/Python/Python310/Scripts/PyTorch/.ipynb_checkpoints/kagglecatsanddogs_5340/PetImages/Dog


100%|██████████| 12501/12501 [00:42<00:00, 296.82it/s]

Cats: 12476
Dogs: 12470


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (24946, 2) + inhomogeneous part.

In [ ]:
import torch 
import torch.nn as nn 
import torch.nn as nn
import torch.nn.functional as F 

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1,32,5)
        self.conv2=nn.Conv2d(1,32,5)
        self.conv3=nn.Conv2d(1,32,5)
        x=torch.randn(50,50).view(-1,1,50,50)
        self.__to__linear=None
        self.convs(x)
    def convs(self,x):
        x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x=F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x=F.max_pool2d(F.relu(self.conv3(x)),(2,2))

        if self.__to__linear is None:
            self.__to__linear=x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
            
        return x
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)  # .view is reshape ... this flattens X before 
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # bc this is our output layer. No activation here.
        return F.softmax(x, dim=1)


net = Net()
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()
X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data]) 
VAL_PCT = 0.1  # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)
print(val_size)
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

BATCH_SIZE = 100
EPOCHS = 1

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
        
        batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
        batch_y = train_y[i:i+BATCH_SIZE]

        net.zero_grad()

        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()    # Does the update

    print("Epoch: "+str(epoch)+' Loss: '+str(loss))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1, 1, 50, 50))[0]  # returns a list, 
        predicted_class = torch.argmax(net_out)

        if predicted_class == real_class:
            correct += 1
        total += 1
print("Accuracy: ", round(correct/total, 3))

tensor([[[[-0.9673, -0.9700,  0.4897,  ...,  1.6895, -0.1393,  1.8951],
          [ 0.3122,  0.8185,  0.4137,  ..., -0.2399, -0.9445, -0.7792],
          [ 0.4117,  0.3847,  1.2028,  ..., -0.4391,  0.3201, -1.7839],
          ...,
          [ 0.1919,  2.0951,  0.4388,  ..., -1.1273,  0.6566,  0.3136],
          [ 0.0673,  0.9557,  0.6894,  ..., -0.0464,  0.1232, -0.3417],
          [ 1.2950,  1.7446, -0.7934,  ...,  0.1061, -0.2514, -0.3972]]]])
